In [18]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 16
### Team Member Names: Matthew Gong, Arya Patel, Saihej Singh
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [55]:
possibleTickers = pd.read_csv("Tickers_Example.csv")
tickers = []
tickers_hist = {}
tickers_long_hist = {}
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
start_date = "2023-01-01"
end_date = "2023-10-31"
long_start_date = "2018-10-31"
threshold_day = 18
days_in_month = []

CAD_stocks = []
USD_stocks = []

for ticker in possibleTickers[possibleTickers.columns.values[0]]:
    currTicker = yf.Ticker(ticker)
    currHist = currTicker.history(start=start_date, end=end_date)
    currFastInfo = currTicker.fast_info

    if currHist.empty:
        print(ticker, "is delisted.")
    elif ticker in tickers:
        print(ticker, "is a duplicate.")
    else:
        tickers_hist[ticker] = currHist
        tickers_long_hist[ticker] = currTicker.history(start=long_start_date, end=end_date)

        group_months = currHist.reset_index().groupby(pd.Grouper(key='Date', freq='M'))
        dfs = [group for _, group in group_months]
        filtered_dfs = []
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))
                filtered_dfs.append(df)
        currHist_updated = pd.concat(filtered_dfs)
        currHist_updated.set_index("Date", inplace=True)

        if currFastInfo["currency"] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif currHist["Volume"].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", currFastInfo["lastVolume"], "and average days of", np.mean(days_in_month))
        else:
            tickers.append(ticker)
            if currFastInfo['currency'] == 'USD':
                USD_stocks.append(ticker)
            elif currFastInfo['currency'] == 'CAD':
                CAD_stocks.append(ticker)
                
            
            
print("CAD Stocks: ", CAD_stocks)
print("USD Stocks: ", USD_stocks)
print("All Valid Stocks: ", tickers)

AGN: No timezone found, symbol may be delisted


AGN is delisted.


CELG: No timezone found, symbol may be delisted


CELG is delisted.


MON: No timezone found, symbol may be delisted


MON is delisted.


RTN: No timezone found, symbol may be delisted


RTN is delisted.
USB is a duplicate.
CAD Stocks:  ['RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO']
USD Stocks:  ['ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']
All Valid Stocks:  ['ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [57]:
stock_stds = []
stocks_df = pd.Dataframe()


for ticker in tickers:
    currTicker = yf.Ticker(ticker)
    currClose = currTicker.history(start = start_date, end = end_date).Close
    currPctChange = currClose.pct_change() * 100
    stock_stds.append(currPctChange.std())
    stocks_df[ticker] = currClose
    

    

    
print(stocks_df)    
print("Stocks STDs:", stock_stds)

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [20]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=long_start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_long_hist[ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in tickers:
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


[('SHOP.TO', 1.8105683661552794),
 ('C', 1.5566636532235243),
 ('BA', 1.4762120488587196),
 ('BAC', 1.3851001463957404),
 ('PYPL', 1.3792816900669551),
 ('BLK', 1.283591011586973),
 ('ACN', 1.230450657671246),
 ('QCOM', 1.2167444394024312),
 ('AXP', 1.2146616027550952),
 ('AMZN', 1.1719942897819302),
 ('UPS', 1.104947577827367),
 ('UNP', 1.1025177948929588),
 ('BK', 1.0953384742317638),
 ('CAT', 1.0879435927701029),
 ('AIG', 1.0351015073852912),
 ('USB', 1.0267301548053611),
 ('TXN', 1.0168285976753872),
 ('PM', 0.7766931774271923),
 ('ABT', 0.6933933901170095),
 ('MO', 0.6535229158839757),
 ('TD.TO', 0.6386755352088388),
 ('UNH', 0.6321134549387507),
 ('PFE', 0.630532058298129),
 ('LMT', 0.5987248186437317),
 ('KO', 0.590810772400325),
 ('PEP', 0.5726422457894019),
 ('RY.TO', 0.5664459640066557),
 ('ABBV', 0.5070220648696108),
 ('PG', 0.47479077827546673),
 ('CL', 0.4606669226867345),
 ('T.TO', 0.45966964639659325),
 ('BMY', 0.3853918172999649),
 ('MRK', 0.36452661157283855),
 ('LLY',

In [21]:
def correlation(stock1, stock2):
    stock1Hist = tickers_hist[stock1].copy()
    stock2Hist = tickers_hist[stock2].copy()

    prices = pd.DataFrame(stock1Hist["Close"])
    prices.columns = [stock1]
    prices[stock2] = stock2Hist["Close"]
    
    monthly_returns = prices.resample('M').ffill().pct_change()
    monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

    return monthly_returns.corr()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.